In [34]:
# Import dependencies

import numpy as np
import rasterio
import os

# Functions

In [75]:
def get_labels(labelpath):
    man_ims = [os.path.join(labelpath, f'manual/{file}') for file in os.listdir(os.path.join(labelpath, f'manual')) if file.endswith('.tif')]
    otsu_ims = [os.path.join(labelpath, f'otsu/{file}') for file in os.listdir(os.path.join(labelpath, f'otsu')) if file.endswith('.tif')]
    kmeans_ims = [os.path.join(labelpath, f'kmeans/{file}') for file in os.listdir(os.path.join(labelpath, f'kmeans')) if file.endswith('.tif')]
    gmm_ims = [os.path.join(labelpath, f'gmm{file}') for file in os.listdir(os.path.join(labelpath, f'gmm')) if file.endswith('.tif')]

    return man_ims, otsu_ims, kmeans_ims, gmm_ims

def get_grd(grdpath):
    orig_ims = [os.path.join(grdpath, f'Original/{file}') for file in os.listdir(os.path.join(grdpath, f'Original')) if file.endswith('.tif')]
    filt_ims = [os.path.join(grdpath, f'GammaMap{file}') for file in os.listdir(os.path.join(grdpath, f'GammaMap')) if file.endswith('.tif')]

    return orig_ims, filt_ims

def get_glcm(glcmpath):
    orig_glcms = [os.path.join(glcmpath, f'Original/{file}') for file in os.listdir(os.path.join(glcmpath, f'Original')) if file.endswith('.tif')]
    filt_glcms = [os.path.join(glcmpath, f'GammaMap{file}') for file in os.listdir(os.path.join(glcmpath, f'GammaMap')) if file.endswith('.tif')]

    return orig_glcms, filt_glcms

# Clip each Sentinel-1 image
def clip_sentinel1_image(s1_path, output_path, s2_bounds):
    with rasterio.open(s1_path) as src:
        # Calculate the window corresponding to the bounding box (extent)
        window = rasterio.windows.from_bounds(*s2_bounds, transform=src.transform)
        
        # Read and clip the Sentinel-1 image
        clipped_image = src.read(window=window)
        
        # Create metadata for the clipped image
        out_meta = src.meta.copy()
        out_meta.update({
            'height': window.height,
            'width': window.width,
            'transform': src.window_transform(window)
        })
        
        # Write the clipped image to a new file
        with rasterio.open(output_path, 'w', **out_meta) as dst:
            dst.write(clipped_image)

def export_s1images(coll, s2_bounds, type):
    """
    type = str
        'Original' or 'GammaMap'
    
    """

    for s1_path in enumerate(coll):
        output_path = os.path.join(s1_path[:16], f'Clipped/{type}/{s1_path[-17:]}')
        clip_sentinel1_image(s1_path, output_path, s2_bounds)

# 1. Clip the Sentinel-1 bands to the same extent as the Sentinel-2 imagery 

In [72]:
# set the directory for where your images are located
man_ims, otsu_ims, kmeans_ims, gmm_ims = get_labels('/mnt/d/SabineMSI/s2classifications')
orig_ims, filt_ims = get_grd('/mnt/d/SabineGRD/GEE')
orig_glcms, filt_glcms = get_glcm('/mnt/d/SabineGRD/GLCM')

In [ ]:
# match the S1 images with their closest labels by date


In [73]:
# Open the Sentinel-2 image (labeled data)
with rasterio.open(man_ims[0]) as src:
    s2_bounds = src.bounds  # Get the bounding box (extent)

In [74]:
# Load the VV or VH band from a Sentinel-1 image_vv
with rasterio.open(orig_ims[90]) as src:
    image_vv = src.read(1)
    image_vh = src.read(2)

# 2a. Register Sentinel-1 Imagery to Sentinel-2 labeled data?
- should be in same reference system, so not sure if they need to be registered

# 2b. Label the Sentinel-1 data as land or water

In [ ]:
# check each labeled image to make sure the Subaerial Land = 1 and Water/Subaqeuous Land = 0


In [ ]:
# Label the S1 pixels as 0 or 1 depending on overlap with labeled image from S2


# 3. Save labeled S1 imagery for training Neural Network

In [ ]:
# save the S1 (original and filtered) with all bands in one image for training NN
# include VV, VH, GLCM, and the four classes for each pixel
